In [0]:
import requests
import base64
import time
from urllib.parse import urlparse, parse_qs, urlencode

In [0]:
CLIENT_ID = "9c22135f704248acb4c79520646559c3"
CLIENT_SECRET = "7d81b48def1e427580328769dd1349df"
REDIRECT_URI = "http://127.0.0.1:5000/callback"

# Definește permisiunile de care are nevoie pipeline-ul tău
SCOPES = (
    "user-read-recently-played "
    "playlist-read-private "
    "user-library-read"
)

AUTH_URL = "https://accounts.spotify.com/authorize"

# Creează widget-ul unde vei lipi URL-ul
dbutils.widgets.text("1_PASTE_URL_HERE", "", "1. Lipește URL-ul de la localhost aici")

def generate_auth_url(client_id, redirect_uri, scopes, auth_url):
    auth_params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": scopes,
    }
    return f"{auth_url}?{urlencode(auth_params)}"

def print_instructions(auth_url):
    print("--- PASUL 1: AUTORIZAREA ---")
    print("Copiază link-ul de mai jos și deschide-l în browser-ul tău local:")
    print("\n")
    print(auth_url)
    print("\n--- PASUL 2: AȘTEPTARE ---")
    print("Autentifică-te, acceptă permisiunile, și vei fi redirecționat către 'localhost' (care va da o eroare).")
    print("Copiază URL-ul complet din bara de adrese a browserului și lipește-l în caseta '1_PASTE_URL_HERE' de mai sus.")

auth_redirect_url = generate_auth_url(CLIENT_ID, REDIRECT_URI, SCOPES, AUTH_URL)
print_instructions(auth_redirect_url)

In [0]:
TOKEN_URL = "https://accounts.spotify.com/api/token"

# --- 1. Preluarea URL-ului din widget ---
redirected_url = dbutils.widgets.get("1_PASTE_URL_HERE")

def get_auth_code(redirected_url):
    parsed_url = urlparse(redirected_url)
    return parse_qs(parsed_url.query).get('code', [None])[0]

def request_refresh_token(auth_code):
    auth_str = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_b64 = base64.b64encode(auth_str.encode()).decode()
    headers = {"Authorization": f"Basic {auth_b64}"}
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": REDIRECT_URI
    }
    response = requests.post(TOKEN_URL, data=data, headers=headers)
    response.raise_for_status()
    return response.json()

def print_success(refresh_token):
    print("\n" + "="*50)
    print("--- ✅ SUCCES! REFRESH TOKEN OBȚINUT ---")
    print("Acesta este noul tău Refresh Token. Copiază-l.")
    print("\n")
    print(refresh_token)
    print("\n" + "="*50)
    print("\n--- ACUM ---")
    print("1. Salvează acest token în Databricks Secrets:")
    print(f"  databricks secrets put-secret spotify_secrets refresh-token --string-value=\"{refresh_token}\"")
    print("2. Șterge acest notebook temporar ('TEMP_Get_Token').")

if not redirected_url:
    print("EROARE: Nu ai lipit URL-ul în caseta '1_PASTE_URL_HERE' de mai sus.")
    print("Rulează din nou Celula 2, fă pașii din browser și lipește URL-ul, apoi rulează această celulă.")
else:
    try:
        # --- 2. Extragerea codului de autorizare ---
        auth_code = get_auth_code(redirected_url)
        if not auth_code:
            raise ValueError("Codul de autorizare nu a fost găsit în URL.")
        print("Cod de autorizare obținut cu succes!")

        # --- 3. Schimbarea codului pentru un REFRESH TOKEN ---
        print("Se solicită Refresh Token-ul...")
        token_info = request_refresh_token(auth_code)

        if "refresh_token" in token_info:
            print_success(token_info['refresh_token'])
        else:
            print("EROARE: Răspunsul de la Spotify NU a inclus un 'refresh_token'.")
            print(token_info)

    except Exception as e:
        print(f"\nEROARE: {e}")
        print("Verifică dacă ai copiat URL-ul *complet* sau dacă CLIENT_ID/CLIENT_SECRET sunt corecte.")